#BUILDING A FEEDFORWARD NEURAL NETWORK WITH PYTORCH
#Model E: TestLoop

This Code has the TestLoop also. Refer the same for writing the test loop for the Homework examples- Model A,B and C

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [6]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295821  [    0/60000]
loss: 2.289475  [ 6400/60000]
loss: 2.262299  [12800/60000]
loss: 2.263804  [19200/60000]
loss: 2.245057  [25600/60000]
loss: 2.209646  [32000/60000]
loss: 2.226006  [38400/60000]
loss: 2.187068  [44800/60000]
loss: 2.179472  [51200/60000]
loss: 2.148252  [57600/60000]
Test Error: 
 Accuracy: 43.5%, Avg loss: 2.144445 

Epoch 2
-------------------------------
loss: 2.152849  [    0/60000]
loss: 2.148293  [ 6400/60000]
loss: 2.082011  [12800/60000]
loss: 2.104573  [19200/60000]
loss: 2.048616  [25600/60000]
loss: 1.981897  [32000/60000]
loss: 2.018650  [38400/60000]
loss: 1.929523  [44800/60000]
loss: 1.933966  [51200/60000]
loss: 1.860213  [57600/60000]
Test Error: 
 Accuracy: 58.0%, Avg loss: 1.859854 

Epoch 3
-------------------------------
loss: 1.891618  [    0/60000]
loss: 1.869959  [ 6400/60000]
loss: 1.736990  [12800/60000]
loss: 1.788543  [19200/60000]
loss: 1.669728  [25600/60000]
loss: 1.619068  [32000/600